In [9]:
import streamlit as st
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pytz
import datetime as datetime
import os
import sys

In [10]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'FPL')))


In [11]:
from FPL.fpl_api_collection import (
    get_bootstrap_data,
    get_current_gw,
    get_fixt_dfs,
    get_league_table
)
from FPL.fpl_utils import (
    define_sidebar,
    get_annot_size,
    map_float_to_color,
    get_text_color_from_hash,
    get_rotation
)
from FPL.fpl_params import (
    TIMEZONES_BY_CONTINENT,
    AUTHOR_CONTINENT,
    AUTHOR_CITY
)

In [12]:
team_fdr_df, team_fixt_df, team_ga_df, team_gf_df = get_fixt_dfs()

events_df = pd.DataFrame(get_bootstrap_data()['events'])

gw_min = min(events_df['id'])
gw_max = max(events_df['id'])

ct_gw = get_current_gw()

In [52]:
sui=team_fixt_df.reset_index()


val=team_fdr_df.reset_index()

In [53]:
val

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,ARS,2.0,4.0,2.0,4.0,5.0,2.0,2.0,3.0,5.0,...,4.0,2.0,2.0,2.0,2.0,2.0,2.0,5.0,3.0,2.0
1,AVL,2.0,5.0,2.0,2.0,2.0,2.0,3.0,3.0,2.0,...,5.0,3.0,3.0,2.0,3.0,5.0,2.0,3.0,4.0,3.0
2,BOU,2.0,3.0,2.0,4.0,5.0,2.0,2.0,5.0,4.0,...,2.0,2.0,2.0,2.0,2.0,3.0,5.0,3.0,5.0,2.0
3,BRE,2.0,5.0,2.0,5.0,4.0,2.0,2.0,3.0,2.0,...,3.0,4.0,4.0,5.0,2.0,2.0,3.0,2.0,2.0,2.0
4,BHA,2.0,3.0,5.0,2.0,3.0,4.0,4.0,4.0,2.0,...,5.0,3.0,2.0,2.0,3.0,2.0,3.0,2.0,5.0,4.0
5,CHE,5.0,2.0,2.0,3.0,2.0,2.0,3.0,5.0,3.0,...,5.0,4.0,3.0,2.0,3.0,2.0,5.0,4.0,3.0,2.0
6,CRY,3.0,2.0,4.0,2.0,3.0,2.0,5.0,2.0,4.0,...,4.0,2.0,2.0,5.0,2.0,5.0,3.0,4.0,2.0,5.0
7,EVE,2.0,4.0,2.0,4.0,2.0,2.0,3.0,2.0,2.0,...,2.0,5.0,5.0,2.0,5.0,4.0,2.0,3.0,2.0,4.0
8,FUL,3.0,2.0,2.0,2.0,3.0,2.0,5.0,3.0,2.0,...,4.0,5.0,5.0,3.0,4.0,2.0,4.0,2.0,3.0,5.0
9,IPS,5.0,5.0,2.0,3.0,2.0,3.0,2.0,2.0,3.0,...,3.0,3.0,2.0,4.0,5.0,4.0,2.0,2.0,2.0,2.0


In [51]:
import pandas as pd
import streamlit as st

# Assuming 'sui' and 'val' DataFrames are already defined and structured correctly

# Prepare for FDR Matrix Calculation
teams = sui[0]  # List of teams from the fixtures DataFrame
num_gameweeks = sui.shape[1] - 1  # Exclude team column
fdr_matrix = pd.DataFrame(index=teams, columns=[f'GW{i}' for i in range(num_gameweeks)])  # Initialize FDR matrix

# Populate FDR Matrix
for index, row in sui.iterrows():
    for gw in range(1, num_gameweeks + 1):
        fixture = row[gw]
        team_a = row[0]  # Home/Away team
        team_h = fixture.split()[0]  # Extracting opponent team name from 'TEAM (H/A)'

        # Retrieve the FDR value from the 'val' DataFrame
        fdr_value = val.loc[team_h, gw - 1] if team_h in val.index else None  # Adjust index to match gameweek

        # Store the FDR value in the matrix
        fdr_matrix.at[team_a, f'GW{gw - 1}'] = fdr_value

# Color Coding Function
def color_fdr(value):
    colors = {
        1: ('#257d5a', 'black'),
        2: ('#00ff86', 'black'),
        3: ('#ebebe4', 'black'),
        4: ('#ff005a', 'white'),
        5: ('#861d46', 'white'),
    }
    bg_color, font_color = colors.get(value, ('white', 'black'))
    return f'background-color: {bg_color}; color: {font_color};'

# Streamlit Display
st.title("Fixture Difficulty Rating (FDR) Matrix")
selected_gameweek = st.sidebar.slider(
    "Select Gameweek:",
    min_value=0,
    max_value=num_gameweeks - 1,
    value=0
)

# Filter FDR Matrix for the selected Gameweek and the next 9 Gameweeks
filtered_fdr_matrix = fdr_matrix.loc[:, f'GW{selected_gameweek}':f'GW{selected_gameweek + 9}'].fillna('')

# Apply Color Coding
styled_fdr_table = filtered_fdr_matrix.style.applymap(color_fdr)

# Display FDR Matrix
st.write(st.markdown(f"**Fixture Difficulty Rating (FDR) for Gameweek {selected_gameweek + 1} onwards:**"))
st.write(styled_fdr_table)
colors = {
            1: ('#257d5a', 'black'),
            2: ('#00ff86', 'black'),
            3: ('#ebebe4', 'black'),
            4: ('#ff005a', 'white'),
            5: ('#861d46', 'white'),
        }
# FDR Legend
st.sidebar.markdown("**Legend:**")
for fdr, (bg_color, font_color) in colors.items():
    st.sidebar.markdown(
        f"<span style='background-color: {bg_color}; color: {font_color}; padding: 2px 5px; border-radius: 3px;'>"
        f"{fdr} - {'Very Easy' if fdr == 1 else 'Easy' if fdr == 2 else 'Medium' if fdr == 3 else 'Difficult' if fdr == 4 else 'Very Difficult'}"
        f"</span>",
        unsafe_allow_html=True,
    )


AttributeError: 'str' object has no attribute 'get'